In [1]:
!pip install Sastrawi
!pip install swifter
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.metrics.distance import edit_distance
nltk.download('stopwords')
import string
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OmenbyHP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#import dataset
df = pd.read_csv('dataset_tweet.csv')
df.head(10)

,Label,Tweet
0,Non_HS,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...
1,Non_HS,RT @baguscondromowo: Mereka terus melukai aksi...
2,Non_HS,Sylvi: bagaimana gurbernur melakukan kekerasan...
3,Non_HS,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja..."
4,Non_HS,RT @lisdaulay28: Waspada KTP palsu.....kawal P...
5,Non_HS,@felixsiauw masih bnyak hal agamis yg bisa dib...
6,Non_HS,Ahok Akan Integrasikan Transportasi Publik den...
7,Non_HS,"Ahok mampu melawan korupsi di ibukota, bangun ..."
8,Non_HS,"Awasi Kecurangan Pilkada, Agus - Sylvi Pakai A..."
9,Non_HS,Bisa gak kita seperti Jokowi dan Ahok? Aku kal...


In [4]:
#Cleaning Text
def cleaning_text(text):
  #text = "".join([char for char in text if char not in string.punctuation])
  text = re.sub(r'@[A-Za-z0-0]+', '', text) #menghilangkan mention
  text = re.sub(r'#[A-Za-z0-0]+', '', text)
  text = re.sub(r'https?:\/\/\S+', '', text) #menghilangkan link
  text = re.sub(r'http?:\/\/\S+', '', text)
  text = re.sub(r'[^a-zA-z0-9]', ' ', str(text))
  text = re.sub(r'\b\w{1,2}\b', '', text) #menghilangkan 2 kata
  text = re.sub(r'\s\s+', ' ', text)
  text = re.sub(r'[@]',r' ',text)
  text = re.sub(r'\d+', '', text)
  text = text.translate(str.maketrans('','',string.punctuation))
  text = re.sub(r'', '', text.strip()) #menghilangkan spasi yang kosong
  return text

df['Tweet'] = df['Tweet'].apply(lambda x: cleaning_text(x))
df.head(10)

,Label,Tweet
0,Non_HS,Fadli Zon Minta Mendagri Segera Menonaktifkan ...
1,Non_HS,Mereka terus melukai aksi dalam rangka memenja...
2,Non_HS,Sylvi bagaimana gurbernur melakukan kekerasan ...
3,Non_HS,Ahmad Dhani Tak Puas Debat Pilkada Masalah Jal...
4,Non_HS,Waspada KTP palsu kawal PILKADA https OOoER...
5,Non_HS,masih bnyak hal agamis bisa dibahas kenapa pil...
6,Non_HS,Ahok Akan Integrasikan Transportasi Publik den...
7,Non_HS,Ahok mampu melawan korupsi ibukota bangun infr...
8,Non_HS,Awasi Kecurangan Pilkada Agus Sylvi Pakai Apli...
9,Non_HS,Bisa gak kita seperti Jokowi dan Ahok Aku kale...


In [5]:
#Tokenization
def tokenization(text):
	text = re.split('\W+', text)
	return text

df['Tokenization'] = df['Tweet'].apply(lambda x: tokenization(x.lower()))
df.head(10)

,Label,Tweet,Tokenization
0,Non_HS,Fadli Zon Minta Mendagri Segera Menonaktifkan ...,"[fadli, zon, minta, mendagri, segera, menonakt..."
1,Non_HS,Mereka terus melukai aksi dalam rangka memenja...,"[mereka, terus, melukai, aksi, dalam, rangka, ..."
2,Non_HS,Sylvi bagaimana gurbernur melakukan kekerasan ...,"[sylvi, bagaimana, gurbernur, melakukan, keker..."
3,Non_HS,Ahmad Dhani Tak Puas Debat Pilkada Masalah Jal...,"[ahmad, dhani, tak, puas, debat, pilkada, masa..."
4,Non_HS,Waspada KTP palsu kawal PILKADA https OOoER...,"[waspada, ktp, palsu, kawal, pilkada, https, o..."
5,Non_HS,masih bnyak hal agamis bisa dibahas kenapa pil...,"[masih, bnyak, hal, agamis, bisa, dibahas, ken..."
6,Non_HS,Ahok Akan Integrasikan Transportasi Publik den...,"[ahok, akan, integrasikan, transportasi, publi..."
7,Non_HS,Ahok mampu melawan korupsi ibukota bangun infr...,"[ahok, mampu, melawan, korupsi, ibukota, bangu..."
8,Non_HS,Awasi Kecurangan Pilkada Agus Sylvi Pakai Apli...,"[awasi, kecurangan, pilkada, agus, sylvi, paka..."
9,Non_HS,Bisa gak kita seperti Jokowi dan Ahok Aku kale...,"[bisa, gak, kita, seperti, jokowi, dan, ahok, ..."


In [6]:
#Stop Removal

stopword = nltk.corpus.stopwords.words('indonesian')

def remove_stopwords(text):
	text = [word for word in text if word not in stopword]
	return text

df['Stop_Removal'] = df['Tokenization'].apply(lambda x: remove_stopwords(x))
df.head(10)

,Label,Tweet,Tokenization,Stop_Removal
0,Non_HS,Fadli Zon Minta Mendagri Segera Menonaktifkan ...,"[fadli, zon, minta, mendagri, segera, menonakt...","[fadli, zon, mendagri, menonaktifkan, ahok, gu..."
1,Non_HS,Mereka terus melukai aksi dalam rangka memenja...,"[mereka, terus, melukai, aksi, dalam, rangka, ...","[melukai, aksi, rangka, memenjarakan, ahok, ah..."
2,Non_HS,Sylvi bagaimana gurbernur melakukan kekerasan ...,"[sylvi, bagaimana, gurbernur, melakukan, keker...","[sylvi, gurbernur, kekerasan, perempuan, bukti..."
3,Non_HS,Ahmad Dhani Tak Puas Debat Pilkada Masalah Jal...,"[ahmad, dhani, tak, puas, debat, pilkada, masa...","[ahmad, dhani, puas, debat, pilkada, jalan, be..."
4,Non_HS,Waspada KTP palsu kawal PILKADA https OOoER...,"[waspada, ktp, palsu, kawal, pilkada, https, o...","[waspada, ktp, palsu, kawal, pilkada, https, o..."
5,Non_HS,masih bnyak hal agamis bisa dibahas kenapa pil...,"[masih, bnyak, hal, agamis, bisa, dibahas, ken...","[bnyak, agamis, dibahas, pilkada, tingkat, kei..."
6,Non_HS,Ahok Akan Integrasikan Transportasi Publik den...,"[ahok, akan, integrasikan, transportasi, publi...","[ahok, integrasikan, transportasi, publik, sis..."
7,Non_HS,Ahok mampu melawan korupsi ibukota bangun infr...,"[ahok, mampu, melawan, korupsi, ibukota, bangu...","[ahok, melawan, korupsi, ibukota, bangun, infr..."
8,Non_HS,Awasi Kecurangan Pilkada Agus Sylvi Pakai Apli...,"[awasi, kecurangan, pilkada, agus, sylvi, paka...","[awasi, kecurangan, pilkada, agus, sylvi, paka..."
9,Non_HS,Bisa gak kita seperti Jokowi dan Ahok Aku kale...,"[bisa, gak, kita, seperti, jokowi, dan, ahok, ...","[gak, jokowi, ahok, kalem, melengkapi]"


In [7]:
#Case Folding
df['Tweet'] = df['Tweet'].str.lower()
df.head(10)

,Label,Tweet,Tokenization,Stop_Removal
0,Non_HS,fadli zon minta mendagri segera menonaktifkan ...,"[fadli, zon, minta, mendagri, segera, menonakt...","[fadli, zon, mendagri, menonaktifkan, ahok, gu..."
1,Non_HS,mereka terus melukai aksi dalam rangka memenja...,"[mereka, terus, melukai, aksi, dalam, rangka, ...","[melukai, aksi, rangka, memenjarakan, ahok, ah..."
2,Non_HS,sylvi bagaimana gurbernur melakukan kekerasan ...,"[sylvi, bagaimana, gurbernur, melakukan, keker...","[sylvi, gurbernur, kekerasan, perempuan, bukti..."
3,Non_HS,ahmad dhani tak puas debat pilkada masalah jal...,"[ahmad, dhani, tak, puas, debat, pilkada, masa...","[ahmad, dhani, puas, debat, pilkada, jalan, be..."
4,Non_HS,waspada ktp palsu kawal pilkada https oooer...,"[waspada, ktp, palsu, kawal, pilkada, https, o...","[waspada, ktp, palsu, kawal, pilkada, https, o..."
5,Non_HS,masih bnyak hal agamis bisa dibahas kenapa pil...,"[masih, bnyak, hal, agamis, bisa, dibahas, ken...","[bnyak, agamis, dibahas, pilkada, tingkat, kei..."
6,Non_HS,ahok akan integrasikan transportasi publik den...,"[ahok, akan, integrasikan, transportasi, publi...","[ahok, integrasikan, transportasi, publik, sis..."
7,Non_HS,ahok mampu melawan korupsi ibukota bangun infr...,"[ahok, mampu, melawan, korupsi, ibukota, bangu...","[ahok, melawan, korupsi, ibukota, bangun, infr..."
8,Non_HS,awasi kecurangan pilkada agus sylvi pakai apli...,"[awasi, kecurangan, pilkada, agus, sylvi, paka...","[awasi, kecurangan, pilkada, agus, sylvi, paka..."
9,Non_HS,bisa gak kita seperti jokowi dan ahok aku kale...,"[bisa, gak, kita, seperti, jokowi, dan, ahok, ...","[gak, jokowi, ahok, kalem, melengkapi]"


In [8]:
# STEMMING
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['Stop_Removal']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['Tweet_Stemmed'] = df['Stop_Removal'].swifter.apply(get_stemmed_term)
print(df['Tweet_Stemmed'])

pintu': 'pintu', 'air': 'air', 'manggarai': 'manggarai', 'jaman': 'jaman', 'gpp': 'gpp', 'belom': 'bom', 'rejekinya': 'rejekinya', 'berkarya': 'karya', 'akitbat': 'akitbat', 'gubernurnya': 'gubernur', 'team': 'team', 'memimpin': 'pimpin', 'banggakan': 'bangga', 'lindungan': 'lindung', 'sejarah': 'sejarah', 'mencatat': 'catat', 'perjuangan': 'juang', 'ditempatkan': 'tempat', 'terhebat': 'hebat', 'inspirasi': 'inspirasi', 'pengalaman': 'alam', 'berharga': 'harga', 'setuju': 'tuju', 'kerena': 'rena', 'keluarga': 'keluarga', 'kerjakan': 'kerja', 'meridhoi': 'meridhoi', 'terpenting': 'penting', 'satunya': 'satu', 'jam': 'jam', 'menjual': 'jual', 'iman': 'iman', 'tulus': 'tulus', 'kerasnya': 'keras', 'asing': 'asing', 'menyorot': 'sorot', 'belajar': 'ajar', 'dipetik': 'petik', 'lupakan': 'lupa', 'hasilkan': 'hasil', 'pastikan': 'pasti', 'sejuk': 'sejuk', 'parpol': 'parpol', 'sulit': 'sulit', 'disiplinkan': 'disiplin', 'massa': 'massa', 'jaga': 'jaga', 'polisi': 'polisi', 'terapkan': 'terap',

In [9]:
df.to_csv("dataset_preprocessed_tweet.csv")